In [297]:
import torch 
import numpy as np 
import torchvision
import torch.nn.functional as F
from torch import nn

from torchvision.transforms import ToTensor
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import random
from torchvision.utils import make_grid

import time
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset, DataLoader

In [298]:
path='../input/logistic-regression/Social_Network_Ads.csv'

In [299]:
class MyDataset(Dataset):
    def __init__(self,path ):
        self.df = pd.read_csv(path)
    def Data(self):
        return self.df
    def userid_remove(self):
        return self.df.iloc[:,1:]
   
    


    def __len__(self):
        return len(self.df)




In [300]:
df=MyDataset(path).userid_remove()
dummy=pd.get_dummies(df,drop_first=True)


In [301]:
y=dummy[['Purchased']].values
X=dummy[["Age","EstimatedSalary", "Gender_Male"]]


In [302]:
X=X/X.max()
X=X.values

In [303]:
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [304]:
X_train=torch.tensor(X_train,dtype=torch.float32)
X_test=torch.tensor(X_test,dtype=torch.float32)
y_train=torch.tensor(y_train,dtype=torch.float32)
y_test=torch.tensor(y_test,dtype=torch.float32)

In [305]:
train_dataset = TensorDataset(X_train,y_train) 

    
    
    

In [306]:
class Logistic_torch(torch.nn.Module):
    def __init__(self,input_dim:int,out_dim:int=1):
        
        super(Logistic_torch,self).__init__()
        self.linear1=torch.nn.Linear(input_dim,out_dim)
        self.sigmoid=torch.nn.Sigmoid()
        self.loss=torch.nn.BCELoss()
        #elf.optimizer=torch.optim.SGD
        #elf.lr=lr
  
    def forward(self,x_train):
        y_pred=self.sigmoid(self.linear1(x_train))
    
        return y_pred
    def calc_accuracy(self,y_pred,y_train):
        true=((y_pred.T>0.5).float()==y_train.T).float().sum()
        acc=(true/len(y_train))*100

        return acc
    def Training(self,y_pred,y_true):
        
        
        loss_value=self.loss(y_pred,y_true)
        
        
        
        return loss_value
    def evaluation(self,x_test,y_test) :
        y_pred=self.forward(X_test)
        loss_value=self.loss(y_pred,y_test)
        test_acc=self.calc_accuracy(y_pred,y_test)
        
        return loss_value,test_acc
        
        
    def fit(self,x_train,y_train,x_test,y_test,num_epochs,lr,opt_func=torch.optim.Adam):
        
        optimizer=opt_func(self.parameters(),lr)
        
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            y_train_pred=self.forward(x_train)
            
            train_loss_value=self.Training(y_train_pred,y_train)
            train_acc=self.calc_accuracy(y_train_pred,y_train)
            train_loss_value.backward()
            optimizer.step()
            
          
            valid_loss,valid_acc=self.evaluation(x_test,y_test)
            
            if epoch%20==0:
                
                print("epoch {} train_loss {:.4f} and valid_loss {:.4f} train_acc {:.4f} test_acc {:.4f}".
                  format(epoch,train_loss_value.detach().numpy(),valid_loss.detach().numpy(),train_acc,valid_acc))
            #print(train_loss_value.detach().numpy(),valid_loss.detach().numpy())
            
            
               
    

       

In [307]:

model=Logistic_torch(X_train.shape[1])
optimizer=torch.optim.SGD(model.parameters(),lr=0.05)

model.fit(X_train,y_train,X_test,y_test,1000,0.01)

In [308]:
from torchmetrics import Accuracy

In [309]:
import tensorflow as tf


In [310]:
x_train_tensor=tf.constant(X_train.detach().numpy(),dtype=tf.float32)
y_train_tensor=tf.constant(y_train.detach().numpy(),dtype=tf.float32)
x_test_tensor=tf.constant(X_train.detach().numpy(),dtype=tf.float32)
y_test_tensor=tf.constant(y_train.detach().numpy(),dtype=tf.float32)

In [311]:
class TensorModel(tf.keras.Model):
    def __init__(self, units=1):
        super(TensorModel, self).__init__()
        self.units = units
  

    def build(self, input_shape):
        self.weight = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.bias = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.math.sigmoid(tf.matmul(inputs, self.weight) + self.bias)
        

In [312]:
model=TensorModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [313]:
history=model.fit(x_train_tensor,y_train_tensor,epochs=100,validation_data=(x_test_tensor,y_test_tensor))

In [314]:

from sklearn.linear_model import LogisticRegression


clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

In [315]:
class MyModel(tf.keras.Model):
    

    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(16, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)

    def call(self, inputs):
        x = self.dense1(inputs)
        x=self.dense2(x)
        return x

model = MyModel()

In [316]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [317]:
model.fit(x_train_tensor,y_train_tensor,epochs=100,validation_data=(x_test_tensor,y_test_tensor))